# DDG: Discrete Holonomy and Curvature Convergence Validation

Purpose: verify Discrete Differential Geometry properties using parallel transport and holonomy around small loops, per the spec in CFT+ULCC Framework Implementation.pdf. Summaries of invariants: Flat patch: holonomy T≈I; Sphere: angle≈K·area with K=1 and K_est→1 as h→0.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
np.set_printoptions(precision=12, suppress=True)
fig_dir = Path("spec-first/artifacts/figures")
fig_dir.mkdir(parents=True, exist_ok=True)


In [ ]:
def lam(x, y):
    r2 = x*x + y*y
    return 2.0 / (1.0 + r2)

def grad_loglam(x, y):
    denom = 1.0 + x*x + y*y
    return np.array([ -2.0*x/denom, -2.0*y/denom ], dtype=float)

def christoffel_conformal(x, y):
    gl = grad_loglam(x, y)
    Gamma = np.zeros((2,2,2), dtype=float)
    # Γ^k_{ij} = δ^k_i ∂_j log λ + δ^k_j ∂_i log λ − δ_{ij} ∂^k log λ
    for i in range(2):
        for j in range(2):
            for k in range(2):
                delta_ki = 1.0 if k == i else 0.0
                delta_kj = 1.0 if k == j else 0.0
                delta_ij = 1.0 if i == j else 0.0
                Gamma[k, j, i] = delta_ki * gl[j] + delta_kj * gl[i] - delta_ij * gl[k]
    return Gamma

def lam_flat(x, y):
    return 1.0

def grad_loglam_flat(x, y):
    return np.array([0.0, 0.0], dtype=float)

def christoffel_flat(x, y):
    return np.zeros((2,2,2), dtype=float)


In [ ]:
def transport_matrix(x, y, dx, dy, christoffel_fn):
    Gamma = christoffel_fn(x, y)
    I = np.eye(2)
    Gx = Gamma[:, :, 0]
    Gy = Gamma[:, :, 1]
    P = I - dx * Gx - dy * Gy
    return P

def loop_holonomy(x0, y0, h, christoffel_fn):
    T = np.eye(2)
    x = x0 - 0.5*h
    y = y0 - 0.5*h
    # +x
    P1 = transport_matrix(x, y, h, 0.0, christoffel_fn)
    x += h
    # +y
    P2 = transport_matrix(x, y, 0.0, h, christoffel_fn)
    y += h
    # -x
    P3 = transport_matrix(x, y, -h, 0.0, christoffel_fn)
    x -= h
    # -y
    P4 = transport_matrix(x, y, 0.0, -h, christoffel_fn)
    T = P4 @ P3 @ P2 @ P1
    return T

def rotation_angle_from_T(T):
    num = T[1,0] - T[0,1]
    den = T[0,0] + T[1,1]
    return float(np.arctan2(num, den))

def estimate_K_from_holonomy(T, area):
    angle = rotation_angle_from_T(T)
    return angle / area


In [ ]:
# Flat patch test
x0, y0 = 0.0, 0.0
h_flat = 0.1
T_flat = loop_holonomy(x0, y0, h_flat, christoffel_flat)
I = np.eye(2)
holonomy_norm = np.linalg.norm(T_flat - I, ord='fro')
print(f"Flat holonomy Frobenius norm ||T - I||_F = {holonomy_norm:.3e}")

# Save a simple visual indicator
plt.figure(figsize=(4, 2.6))
plt.axis('off')
plt.text(0.5, 0.6, 'Flat patch holonomy', ha='center', va='center', fontsize=12)
plt.text(0.5, 0.35, f'||T-I||_F = {holonomy_norm:.3e}', ha='center', va='center', fontsize=12)
plt.tight_layout()
plt.savefig(fig_dir / 'ricci_step_flat_holonomy.png', dpi=160)
plt.close()


In [ ]:
# Sphere patch convergence at north-pole chart center (0,0)
x0, y0 = 0.0, 0.0
hs = np.array([0.5, 0.25, 0.125, 0.0625], dtype=float)
K_true = 1.0
lam0 = lam(x0, y0)
K_est_list = []
errors = []
angles = []
for h in hs:
    T_h = loop_holonomy(x0, y0, float(h), christoffel_conformal)
    angle_h = rotation_angle_from_T(T_h)
    # Use metric area at the loop center: a ≈ λ(x0,y0)^2 h^2
    area_h = (lam0*lam0) * (h*h)
    K_est_h = estimate_K_from_holonomy(T_h, area_h)
    K_est_list.append(K_est_h)
    angles.append(angle_h)
    errors.append(abs(K_est_h - K_true))
K_est_list = np.array(K_est_list, dtype=float)
errors = np.array(errors, dtype=float)

print('h values:', hs)
print('Estimated K:', K_est_list)
print('Absolute errors:', errors)

fig, ax = plt.subplots(1, 2, figsize=(10, 3.2))
ax[0].plot(hs, K_est_list, 'o-', label='K_est')
ax[0].axhline(1.0, color='k', lw=1, ls='--', label='K=1 (sphere)')
ax[0].set_xlabel('h (coordinate side length)')
ax[0].set_ylabel('Estimated Gaussian curvature')
ax[0].set_title('K_est vs h')
ax[0].legend(loc='best')

ax[1].loglog(hs, errors + 1e-16, 'o-', label='|K_est - 1|')
ax[1].set_xlabel('h')
ax[1].set_ylabel('abs error')
ax[1].set_title('Convergence (log-log)')
ax[1].legend(loc='best')
plt.tight_layout()
plt.savefig(fig_dir / 'ricci_step_convergence.png', dpi=160)
plt.close()


Results summary and acceptance checks.
We expect on flat patch T≈I and on the sphere K_est→1 as h→0. This relies on a small-step linearization of parallel transport; shrinking h improves the approximation in line with DDG mimetic goals.


In [ ]:
# Acceptance assertions
print(f"Flat case: ||T-I||_F = {holonomy_norm:.3e}")
print('Sphere convergence table:')
print(' h        K_est        abs error')
for h, k, e in zip(hs, K_est_list, errors):
    print(f" {h:0.6f}  {k:0.8f}   {e:0.8f}")

# 1) Flat holonomy must be nearly identity
assert holonomy_norm < 1e-6, 'Flat patch holonomy is not sufficiently close to identity.'

# 2) Errors should be monotone nonincreasing as h halves
diffs = np.diff(errors)
assert np.all(diffs <= 1e-12), 'Curvature estimate errors are not monotonically nonincreasing.'

# 3) Final accuracy tolerance at smallest h
assert errors[-1] < 5e-2, 'Final curvature estimate is not within tolerance.'
print('All acceptance checks passed.')
